# Mission statement #

Want to build a neural network predicting the outcome of a match, given the hero matchup (5 radiant heroes and 5 dire)

In [1]:
import requests
import pprint
import json
import pandas as pd
import numpy as np

Get the hero dictionary:

In [3]:
h = requests.get("https://api.opendota.com/api/heroes")
heroes = json.loads(h.text)
hero_df = pd.DataFrame(heroes)

# Mine the required data. #

We want to get high-mmr matches, using OpenDota API which gives a random sample of 100 recent games. Work iteratively - per every call, sort out the 100 matches, get 100 older matches and repeat.

In [189]:
def pickaxe(below_match_id = None):
    # Pick out relevant matches from a sample of 100
    if below_match_id == None:
        string = 'https://api.opendota.com/api/publicMatches'
    else:
        string = 'https://api.opendota.com/api/publicMatches?less_than_match_id=' + below_match_id 
    a = requests.get(string)
    A = json.loads(a.text)
    over4k = []
    over5k = []
    for i in range(len(A)):
        if A[i]['avg_mmr'] != None:
            if A[i]['game_mode'] == 22 and A[i]['avg_mmr'] > 4000 and A[i]['avg_mmr'] < 5000:
                over4k.append(A[i])
            elif A[i]['game_mode'] == 2 and A[i]['avg_mmr'] > 5000:
                over5k.append(A[i]) 
    lowest_match_id = A[-1]['match_id']
    return over4k, over5k, lowest_match_id

def picked_heroes(hero_string):
    # Convert the string from pickaxe() to a length 120 array with 1s on picked heroes, 0 elsewhere
    team_input = np.zeros(len(hero_df))
    y = [int(i) for i in hero_string.split(',')]
    for item in y:
        idx = int(hero_df[hero_df['id'] == item].index[0])
        team_input[idx] += 1
    return team_input

def process_the_pickaxe(game_list):
    Xt = np.zeros(232)
    yt = np.array(0)
    for item in game_list:
        X = np.hstack((picked_heroes(item['radiant_team']), picked_heroes(item['dire_team'])))
        y = np.array(0)
        y += item['radiant_win']*1
        Xt = np.vstack((Xt,X))
        yt = np.vstack((yt,y))
    Xt = np.delete(Xt, 0, 0)
    yt = np.delete(yt, 0, 0)
    return Xt, yt

#### Playground:####

In [190]:
game_list = pickaxe()[0]
a,b = process_the_pickaxe(game_list)

In [191]:
a.shape

(10, 232)

In [192]:
b.shape

(10, 1)

In [150]:
a.shape

(8, 232)

In [156]:
a.T[0]

array([0., 0., 0., 0., 0., 0., 0., 0.])

In [72]:
a,b,c = pickaxe()

In [73]:
len(a)

8

In [74]:
type(a[0])

dict

In [75]:
a[0]

{'match_id': 4409775007,
 'match_seq_num': 3810074777,
 'radiant_win': False,
 'start_time': 1549820324,
 'duration': 1479,
 'avg_mmr': 4758,
 'num_mmr': 5,
 'lobby_type': 7,
 'game_mode': 22,
 'avg_rank_tier': 72,
 'num_rank_tier': 8,
 'cluster': 133,
 'radiant_team': '19,54,55,121,49',
 'dire_team': '36,71,91,48,108'}